In [2]:
#Program for feature extraction
import nltk

In [3]:
def remove_punctuation(string):              #This function is used to remove punctuation from string
    clearstring=""
    if(string.isalnum()==False):             #This statement returns False if the string has punctuation
        for j in range (len(string)):        #Now we are checking if every letter in the string is a punctuation or not
            if (string[j].isalnum()==False):
                if(j==len(string)-1):
                    if string[j]=="!":
                        clearstring=str(clearstring)+string[j]
                continue
            else:
                clearstring=str(clearstring)+string[j]  #It concatenate only the alphanumeric words
        return clearstring
    else:
        return string

from nltk.corpus import brown
wordlist=list(brown.words(categories='editorial'))   #storing all words from dataset 
words=[remove_punctuation(w) for w in wordlist]      #removing punctuation uinf function
words_unique=set(words)                              #extracting only the unique words
ref_words=[w for w in words_unique if len(w)>1]      #taking words whose length is greater than 1

In [12]:
def features(word):
    return{
    'last_letter':word[-1],     
    'noun': (word[0].upper()==word[0] and word[1:]==word[1:].lower()) or word[-4:]in["ment","ness","ence","ance","hood","ship"] or word[-3:]=="ity",
    'plural': word[len(word)-1]=='s',
    'interjunction':word[-1]=='!',
    'adverb':word[-2:]=="ly",
    'number':str(word).isdigit(),
    'wh_word':word[:2]=='wh',
    'article': word in ["an","the","a"], #Checks if the word is an article
    'gerund':word[-3:]=="ing",
    'verb_past':word[-2:]=="ed",
    'proper_adjective':word[-2:]=="an",
    'be_verb':word in ["am","is","are","was","were"],
    'reflexive_pronoun':word[-4:]=="self" or word[-6:]=="selves",
    'possessive pronoun':word[-2:]=="rs",      #Checks if last two letters end in "rs"
    'subject_pronoun':word in ["I","you","he","she","it","we","you","they"],
    'coordinating_conjunction':word in ["and","but","for","or","not","yet","so"],
    'all_lower':str(word).lower()==word,  #Checks if all letters in word are in lowercase
    'all_upper':str(word).upper()==word,  ##Checks if all letters in word are in uppercase
    }

In [6]:
tag_words=brown.tagged_words()
tag_words1=list(set(tag_words))

In [7]:
def untag(tag_word):
    return [w for w,x in tag_words1]

In [8]:
def transform_to_dataset(list_words):
    X,y=[],[]
    for word in list_words:
        X.append(features(untag(word)))
        y.append(word[1])
    return X,y

In [9]:
cut=int(0.75*len(tag_words1))
training_words = tag_words1[:cut]
testing_words = tag_words1[cut:]

In [10]:
print(len(training_words))
print(len(testing_words))

50204
16735


In [13]:
X,y=transform_to_dataset(training_words[:500])
print(len(X))
print(len(y))

500
500


In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

In [15]:
classifier=Pipeline([
    ('vectorizer',DictVectorizer(sparse=False)),
    ('classifier',DecisionTreeClassifier(criterion='entropy'))
])
classifier.fit(X,y)

Pipeline(memory=None,
         steps=[('vectorizer',
                 DictVectorizer(dtype=<class 'numpy.float64'>, separator='=',
                                sort=True, sparse=False)),
                ('classifier',
                 DecisionTreeClassifier(class_weight=None, criterion='entropy',
                                        max_depth=None, max_features=None,
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort=False, random_state=None,
                                        splitter='best'))],
         verbose=False)